---
## Python Packages & Directories
---

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

import numpy as np
import scipy as sp
import pandas as pd

import seaborn as sns
sns.set_palette("colorblind")
colors = sns.color_palette("colorblind", 20)

# from smh import Session

from spag.read_data import *
from spag.convert import *
from spag.utils import *
from spag.read_data import get_solar
import spag.coordinates as coord

# import alexmods.read_data as rd

script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"
data_dir = '/Users/ayelland/Research/metal-poor-stars/spag/spag/data/'


        Use @ instead. [gala.coordinates.sgr]
        Use @ instead. [gala.coordinates.orphan]
        Use @ instead. [gala.coordinates.magellanic_stream]


In [2]:
## Show all columns and rows of the dataframe
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

---
## Check for Duplicates using Coordinates
---
Here, we will compare twoo datasets to see if there are any stars in common. To do so, we will find common RA and DEC values of the stars.

### Load the Datasets

In [3]:
## Datasets for Comparison
df1 = load_placco2014()
df2 = load_atari()

### Create the comparison Dataframes

In [4]:
## Create an empty DataFrame with the desired columns
coord_df1 = pd.DataFrame(columns=['ID', 'RA_hms', 'DEC_dms', 'RA_deg', 'DEC_deg'])

## Drop rows with NaN values in RA or DEC
df1a = df1.dropna(subset=['RA_hms', 'DEC_dms'])

## Create a list to store rows before converting to DataFrame
tmp_list = []

for index, row in df1a.iterrows():
    name = row['Simbad_Identifier'] if 'Simbad_Identifier' in row else row['Name']

    # Convert RA to degrees/hms
    if "RA_hms" in row.index:
        ra_hms = row['RA_hms'].replace(" ", ":")
        ra_deg = float(coord.ra_hms_to_deg(ra_hms))
    else:
        ra_deg = float(row['RA_deg'])
        ra_hms = coord.ra_deg_to_hms(ra_deg)

    # Convert DEC to degrees/dms
    if "DEC_dms" in row.index:
        dec_hms = row['DEC_dms'].replace(" ", ":")
        dec_deg = float(coord.dec_dms_to_deg(dec_hms))
    else:
        dec_deg = float(row['DEC_deg'])
        dec_hms = coord.dec_deg_to_dms(dec_deg)

    # Append row data to the list
    tmp_list.append({'ID': name, 'RA_hms': ra_hms, 'DEC_dms': dec_hms, 'RA_deg': ra_deg, 'DEC_deg': dec_deg})

# Convert list to DataFrame and concatenate with coord_df1
coord_df1 = pd.concat([coord_df1, pd.DataFrame(tmp_list)], ignore_index=True)
display(coord_df1.head(20))


,ID,RA_hms,DEC_dms,RA_deg,DEC_deg
0,BD +02 3375,17:39:45.5948,+02:24:59.606,264.939978,2.416557
1,BD +03 740,05:01:16.6226,+04:06:37.021,75.319261,4.110284
2,BD +06 0648,04:13:13.1126,+06:36:01.768,63.304636,6.600491
3,BD +10 2495,12:59:19.9306,+09:14:35.634,194.833044,9.243232
4,BD +17 3248,17:28:14.4690,+17:30:35.847,262.060287,17.509957
5,BD +23 3130,17:32:41.6247,+23:44:11.625,263.173436,23.736563
6,BD +24 1676,07:30:41.2662,+24:05:10.248,112.671943,24.086180
7,BD +44 493,02:26:49.7390,+44:57:46.523,36.707246,44.962923
8,BD -01 2582,11:53:37.3231,-02:00:36.728,178.405513,-2.010202
9,BD -04 3208,12:07:15.0721,-05:44:01.612,181.812800,-5.733781


In [5]:
## Create an empty DataFrame with the desired columns
coord_df2 = pd.DataFrame(columns=['ID', 'RA_hms', 'DEC_dms', 'RA_deg', 'DEC_deg'])

## Drop rows with NaN values in RA or DEC
df2a = df2.dropna(subset=['RA_hms', 'DEC_dms'])

## Create a list to store rows before converting to DataFrame
tmp_list = []

for index, row in df2a.iterrows():
    name = row['Simbad_Identifier'] if 'Simbad_Identifier' in row else row['Name']

    # Convert RA to degrees/hms
    if "RA_hms" in row.index:
        ra_hms = row['RA_hms'].replace(" ", ":")
        ra_deg = float(coord.ra_hms_to_deg(ra_hms))
    else:
        ra_deg = float(row['RA_deg'])
        ra_hms = coord.ra_deg_to_hms(ra_deg)

    # Convert DEC to degrees/dms
    if "DEC_dms" in row.index:
        dec_hms = row['DEC_dms'].replace(" ", ":")
        dec_deg = float(coord.dec_dms_to_deg(dec_hms))
    else:
        dec_deg = float(row['DEC_deg'])
        dec_hms = coord.dec_deg_to_dms(dec_deg)

    # Append row data to the list
    tmp_list.append({'ID': name, 'RA_hms': ra_hms, 'DEC_dms': dec_hms, 'RA_deg': ra_deg, 'DEC_deg': dec_deg})

# Convert list to DataFrame and concatenate with coord_df1
coord_df2 = pd.concat([coord_df2, pd.DataFrame(tmp_list)], ignore_index=True)
display(coord_df2.head(20))


,ID,RA_hms,DEC_dms,RA_deg,DEC_deg
0,2MASS J16285613-1014576,16:28:56.15,-10:14:57.10,247.233958,-10.249194
1,2MASS J18284356-8441346,18:28:43.44,-84:41:34.81,277.181000,-84.693003
2,TYC 7602-1143-1,05:52:15.78,-39:53:18.47,88.065750,-39.888464
3,2MASS J14101587-0343553,14:10:15.84,-03:43:55.20,212.566000,-3.732000
4,2MASS J00255030-4808270,00:25:50.30,-48:08:27.07,6.459583,-48.140853
5,2MASS J02215557-5410143,02:21:55.60,-54:10:14.40,35.481667,-54.170667
6,CD-26 16470,23:05:50.54,-25:57:22.29,346.460583,-25.956192
7,HD119516,13:43:26.70,+15:34:31.10,205.861250,15.575306
8,2MASS J15542729+0021368,15:54:27.29,+00:21:36.90,238.613708,0.360250
9,HD23798,03:46:45.72,-30:51:13.32,56.690500,-30.853700


### Perform the comparison

In [6]:
## Compare the coordinates of the Jinabase and LVDB datasets
common_dw_stars = {"Name1": [], "Name2": []}

for idx1, ra1 in coord_df1['RA_deg'].items():
    for idx2, ra2 in coord_df2['RA_deg'].items():

        # Compare RA and DEC values
        if np.isclose(ra1, ra2, atol=0.01):
            dec1 = coord_df1.loc[idx1, 'DEC_deg']
            dec2 = coord_df2.loc[idx2, 'DEC_deg']
            if np.isclose(dec1, dec2, atol=0.01):

                # Save the IDs of the common stars
                name1 = coord_df1.loc[idx1, 'ID']
                name2 = coord_df2.loc[idx2, 'ID']

                common_dw_stars["Name1"].append(name1)
                common_dw_stars["Name2"].append(name2)

                # Print the common stars
                print("ID:  ", name1,  " || ", name2)
                print("RA:  ", ra1,  " || ", ra2)
                print("DEC: ", dec1, " || ", dec2)
                print()
            


ID:   BPS BS 16928-053  ||  2MASS J12222802+3411318
RA:   185.61676666666668  ||  185.61675000000002
DEC:  34.19213611111111  ||  34.19213611111111

ID:   BPS CS 22186-023  ||  2MASS J04194553-3651359
RA:   64.93972166666666  ||  64.93974999999999
DEC:  -36.859994444444446  ||  -36.85997777777778

ID:   BPS CS 22948-104  ||  2MASS J21514574-3752308
RA:   327.94066833333335  ||  327.94058333333334
DEC:  -37.87524305555556  ||  -37.87524444444445

ID:   BPS CS 22960-064  ||  2MASS J22240014-4235160
RA:   336.00059541666667  ||  336.0005833333333
DEC:  -42.587785000000004  ||  -42.58779166666667

ID:   BPS CS 29506-007  ||  BPS CS29506-0007
RA:   320.1193920833333  ||  320.119375
DEC:  -20.773043055555554  ||  -20.773027777777777

ID:   BPS CS 30306-132  ||  2MASS J15141890+0727028
RA:   228.57882958333332  ||  228.57874999999999
DEC:  7.450744444444445  ||  7.450777777777778

ID:   BPS CS 31078-018  ||  BPS CS31079-0028
RA:   45.25286083333334  ||  45.252916666666664
DEC:  6.275502222222

In [7]:
print('df1: ', common_dw_stars["Name1"])
print('df2: ', common_dw_stars["Name2"])

## Display the common stars in df1 and df2
name_key = 'Simbad_Identifier' if 'Simbad_Identifier' in df1.columns else 'Name'
df1_common = df1[df1[name_key].isin(common_dw_stars["Name1"])]
df2_common = df2[df2[name_key].isin(common_dw_stars["Name2"])]

print("df1 common stars: ", len(df1_common))
print("df2 common stars: ", len(df2_common))
display(df1_common)
display(df2_common)

df1:  ['BPS BS 16928-053', 'BPS CS 22186-023', 'BPS CS 22948-104', 'BPS CS 22960-064', 'BPS CS 29506-007', 'BPS CS 30306-132', 'BPS CS 31078-018', 'HD 2796', 'HD 23798', 'HD 119516', 'HE 0017-4346', 'HE 0023-4825', 'HE 1116-0634', 'HE 1300+0157', 'HE 1317-0407', 'HE 1413-1954', 'HE 1424-0241', 'HE 2259-3407', 'HE 2318-1621', 'SDSS J102915.15+172927.9', 'SDSS J124502.68-073847.1']
df2:  ['2MASS J12222802+3411318', '2MASS J04194553-3651359', '2MASS J21514574-3752308', '2MASS J22240014-4235160', 'BPS CS29506-0007', '2MASS J15141890+0727028', 'BPS CS31079-0028', 'HD2796', 'HD23798', 'HD119516', 'HE 0017-4346', '2MASS J00255030-4808270', 'TYC 4928-1438-1', 'HE 1300+0157', 'TYC 4961-1053-1', 'HE 1413-1954', 'HE 1424-0241', '2MASS J23021574-3351110', 'HE 2318-1621', 'SDSS J102915.14+172927.9', 'SDSS J124502.68-073847.0']
df1 common stars:  21
df2 common stars:  21


,Name,Simbad_Identifier,RA_hms,DEC_dms,Teff,logg,log_L,[Fe/H],ul[N/Fe],[N/Fe],[C/Fe],[C/Fe]c,[C/Fe]f,ul[Sr/Fe],[Sr/Fe],ul[Ba/Fe],[Ba/Fe],I/O,Reference,Ref,epsc_c,MP_key,Ncap_key,C_key,epsc,epsc_f,[C/H],[C/H]f,[Sr/H],ul[Sr/H],[Ba/H],ul[Ba/H],[Sr/Ba],ul[Sr/Ba]
30,BS 16928-053,BPS BS 16928-053,12:22:28.0240,+34:11:31.690,4679,1.16,2.82,-2.93,NaN,1.05,-0.24,0.69,0.45,NaN,-0.19,NaN,-0.79,1,Yong+2013,YON13,0.69,VMP,,,5.26,5.95,-3.17,-2.48,-3.12,NaN,-3.72,NaN,0.60,NaN
55,CS 22186-023,BPS CS 22186-023,04:19:45.5332,-36:51:35.980,5066,2.19,1.92,-2.72,NaN,NaN,0.26,0.01,0.27,NaN,-0.05,NaN,-0.98,1,Barklem+2005,BAR05,0.01,VMP,,,5.97,5.98,-2.46,-2.45,-2.77,NaN,-3.70,NaN,0.93,NaN
98,CS 22948-104,BPS CS 22948-104,21:51:45.7604,-37:52:30.875,5000,2.30,1.79,-2.76,NaN,NaN,0.38,0.01,0.39,NaN,NaN,NaN,NaN,1,Masseron+2012,MAS12,0.01,VMP,,,6.05,6.06,-2.38,-2.37,NaN,NaN,NaN,NaN,NaN,NaN
116,CS 22960-064,BPS CS 22960-064,22:24:00.1429,-42:35:16.026,5060,2.20,1.91,-2.77,NaN,1.35,0.14,0.01,0.15,NaN,NaN,NaN,-0.20,1,Roederer+2014,ROE14,0.01,VMP,,,5.80,5.81,-2.63,-2.62,NaN,NaN,-2.97,NaN,NaN,NaN
141,CS 29506-007,BPS CS 29506-007,21:20:28.6541,-20:46:22.955,6522,3.94,0.61,-2.73,0.93,NaN,0.49,0.00,0.49,NaN,0.18,NaN,0.09,1,Yong+2013,YON13,0.00,VMP,,,6.19,6.19,-2.24,-2.24,-2.55,NaN,-2.64,NaN,0.09,NaN
158,CS 30306-132,BPS CS 30306-132,15:14:18.9191,+07:27:02.680,5047,2.16,1.95,-2.56,NaN,NaN,0.34,0.01,0.35,NaN,0.18,NaN,0.22,1,Yong+2013,YON13,0.01,VMP,,,6.21,6.22,-2.22,-2.21,-2.38,NaN,-2.34,NaN,-0.04,NaN
187,CS 31078-018,BPS CS 31078-018,03:01:00.6866,+06:16:31.808,5100,2.27,1.86,-2.99,NaN,-0.38,0.37,0.01,0.38,NaN,0.26,NaN,0.38,1,Yong+2013,YON13,0.01,VMP,,,5.81,5.82,-2.62,-2.61,-2.73,NaN,-2.61,NaN,-0.12,NaN
193,HD 2796,HD 2796,00:31:16.9147,-16:47:40.790,4923,1.84,2.22,-2.31,NaN,0.85,-0.50,0.17,-0.33,NaN,0.33,NaN,-0.14,1,Yong+2013,YON13,0.17,VMP,,,5.62,5.79,-2.81,-2.64,-1.98,NaN,-2.45,NaN,0.47,NaN
200,HD 23798,HD 23798,03:46:45.7220,-30:51:13.333,4450,1.06,2.83,-2.26,NaN,NaN,-0.59,0.74,0.15,NaN,NaN,NaN,NaN,1,Simmerer+2004,SIM04,0.74,VMP,,,5.58,6.32,-2.85,-2.11,NaN,NaN,NaN,NaN,NaN,NaN
211,HD 119516,HD 119516,13:43:26.7251,+15:34:31.069,5382,2.47,1.75,-2.11,NaN,NaN,-1.19,0.01,-1.18,NaN,NaN,NaN,NaN,1,Simmerer+2004,SIM04,0.01,VMP,,,5.13,5.14,-3.30,-3.29,NaN,NaN,NaN,NaN,NaN,NaN


,JINA_ID,Simbad_Identifier,RA_hms,DEC_dms,[Fe/H],[C/Fe],ul[C/Fe],ecc,Z_max,Reference,Name,Reference_2,Ref_2,Ref,Ncap_key,C_key,MP_key,Loc,System,RA_deg,DEC_deg,Priority,NLTE,alpha_key,Type,Vel,Vel_bibcode,U_mag,B_mag,V_mag,R_mag,I_mag,J_mag,H_mag,K_mag,Teff,logg,Fe/H,Vmic,Added_by,epsli,epsbe,epsc,epsn,epso,epsf,epsna,epsmg,epsal,epssi,epsp,epss,epsk,epsca,epssc,epsti,epsv,epscr,epsmn,epsfe,epsco,epsni,epscu,epszn,epsga,epsge,epsas,epsse,epsrb,epssr,epsy,epszr,epsnb,epsmo,epsru,epsrh,epspd,epsag,epscd,epssn,epste,epsba,epsla,epsce,epspr,epsnd,epssm,epseu,epsgd,epstb,epsdy,epsho,epser,epstm,epsyb,epslu,epshf,epsw,epsos,epsir,epspt,epsau,epshg,epspb,epsbi,epsth,epsu,ulli,ulbe,ulc,uln,ulo,ulf,ulna,ulmg,ulal,ulsi,ulp,uls,ulk,ulca,ulsc,ulti,ulv,ulcr,ulmn,ulfe,ulco,ulni,ulcu,ulzn,ulga,ulge,ulas,ulse,ulrb,ulsr,uly,ulzr,ulnb,ulmo,ulru,ulrh,ulpd,ulag,ulcd,ulsn,ulte,ulba,ulla,ulce,ulpr,ulnd,ulsm,uleu,ulgd,ultb,uldy,ulho,uler,ultm,ulyb,ullu,ulhf,ulw,ulos,ulir,ulpt,ulau,ulhg,ulpb,ulbi,ulth,ulu,[Li/H],[Be/H],[C/H],[N/H],[O/H],[F/H],[Na/H],[Mg/H],[Al/H],[Si/H],[P/H],[S/H],[K/H],[Ca/H],[Sc/H],[Ti/H],[V/H],[Cr/H],[Mn/H],[Co/H],[Ni/H],[Cu/H],[Zn/H],[Ga/H],[Ge/H],[As/H],[Se/H],[Rb/H],[Sr/H],[Y/H],[Zr/H],[Nb/H],[Mo/H],[Ru/H],[Rh/H],[Pd/H],[Ag/H],[Cd/H],[Sn/H],[Te/H],[Ba/H],[La/H],[Ce/H],[Pr/H],[Nd/H],[Sm/H],[Eu/H],[Gd/H],[Tb/H],[Dy/H],[Ho/H],[Er/H],[Tm/H],[Yb/H],[Lu/H],[Hf/H],[W/H],[Os/H],[Ir/H],[Pt/H],[Au/H],[Hg/H],[Pb/H],[Bi/H],[Th/H],[U/H],ul[Li/H],ul[Be/H],ul[C/H],ul[N/H],ul[O/H],ul[F/H],ul[Na/H],ul[Mg/H],ul[Al/H],ul[Si/H],ul[P/H],ul[S/H],ul[K/H],ul[Ca/H],ul[Sc/H],ul[Ti/H],ul[V/H],ul[Cr/H],ul[Mn/H],ul[Fe/H],ul[Co/H],ul[Ni/H],ul[Cu/H],ul[Zn/H],ul[Ga/H],ul[Ge/H],ul[As/H],ul[Se/H],ul[Rb/H],ul[Sr/H],ul[Y/H],ul[Zr/H],ul[Nb/H],ul[Mo/H],ul[Ru/H],ul[Rh/H],ul[Pd/H],ul[Ag/H],ul[Cd/H],ul[Sn/H],ul[Te/H],ul[Ba/H],ul[La/H],ul[Ce/H],ul[Pr/H],ul[Nd/H],ul[Sm/H],ul[Eu/H],ul[Gd/H],ul[Tb/H],ul[Dy/H],ul[Ho/H],ul[Er/H],ul[Tm/H],ul[Yb/H],ul[Lu/H],ul[Hf/H],ul[W/H],ul[Os/H],ul[Ir/H],ul[Pt/H],ul[Au/H],ul[Hg/H],ul[Pb/H],ul[Bi/H],ul[Th/H],ul[U/H],[Li/Fe],[Be/Fe],[N/Fe],[O/Fe],[F/Fe],[Na/Fe],[Mg/Fe],[Al/Fe],[Si/Fe],[P/Fe],[S/Fe],[K/Fe],[Ca/Fe],[Sc/Fe],[Ti/Fe],[V/Fe],[Cr/Fe],[Mn/Fe],[Co/Fe],[Ni/Fe],[Cu/Fe],[Zn/Fe],[Ga/Fe],[Ge/Fe],[As/Fe],[Se/Fe],[Rb/Fe],[Sr/Fe],[Y/Fe],[Zr/Fe],[Nb/Fe],[Mo/Fe],[Ru/Fe],[Rh/Fe],[Pd/Fe],[Ag/Fe],[Cd/Fe],[Sn/Fe],[Te/Fe],[Ba/Fe],[La/Fe],[Ce/Fe],[Pr/Fe],[Nd/Fe],[Sm/Fe],[Eu/Fe],[Gd/Fe],[Tb/Fe],[Dy/Fe],[Ho/Fe],[Er/Fe],[Tm/Fe],[Yb/Fe],[Lu/Fe],[Hf/Fe],[W/Fe],[Os/Fe],[Ir/Fe],[Pt/Fe],[Au/Fe],[Hg/Fe],[Pb/Fe],[Bi/Fe],[Th/Fe],[U/Fe],ul[Li/Fe],ul[Be/Fe],ul[N/Fe],ul[O/Fe],ul[F/Fe],ul[Na/Fe],ul[Mg/Fe],ul[Al/Fe],ul[Si/Fe],ul[P/Fe],ul[S/Fe],ul[K/Fe],ul[Ca/Fe],ul[Sc/Fe],ul[Ti/Fe],ul[V/Fe],ul[Cr/Fe],ul[Mn/Fe],ul[Co/Fe],ul[Ni/Fe],ul[Cu/Fe],ul[Zn/Fe],ul[Ga/Fe],ul[Ge/Fe],ul[As/Fe],ul[Se/Fe],ul[Rb/Fe],ul[Sr/Fe],ul[Y/Fe],ul[Zr/Fe],ul[Nb/Fe],ul[Mo/Fe],ul[Ru/Fe],ul[Rh/Fe],ul[Pd/Fe],ul[Ag/Fe],ul[Cd/Fe],ul[Sn/Fe],ul[Te/Fe],ul[Ba/Fe],ul[La/Fe],ul[Ce/Fe],ul[Pr/Fe],ul[Nd/Fe],ul[Sm/Fe],ul[Eu/Fe],ul[Gd/Fe],ul[Tb/Fe],ul[Dy/Fe],ul[Ho/Fe],ul[Er/Fe],ul[Tm/Fe],ul[Yb/Fe],ul[Lu/Fe],ul[Hf/Fe],ul[W/Fe],ul[Os/Fe],ul[Ir/Fe],ul[Pt/Fe],ul[Au/Fe],ul[Hg/Fe],ul[Pb/Fe],ul[Bi/Fe],ul[Th/Fe],ul[U/Fe],epsti1,epsfe2,ulti1,ulfe2,[TiI/H],[FeII/H],ul[TiI/H],ul[FeII/H],[TiI/Fe],ul[TiI/Fe],e_c,e_o,e_na,e_mg,e_al,e_si,e_ca,e_sc,e_ti1,e_ti,e_cr,e_mn,e_fe,e_fe2,e_co,e_ni,e_zn,e_sr,e_ba
4,71.0,2MASS J00255030-4808270,00:25:50.30,-48:08:27.07,-2.06,0.27,NaN,0.53,1.61,Mardini+2022a,2MASS J00255030-4808270,Barklem+2005,BAR05,MAR22a,,,VMP,DW,Atari Disk,6.459583,-48.140853,1.0,NaN,NaN,SG,-21.50,2005A&A...439..129B,NaN,14.298,13.830,13.485,13.161,12.696,12.393,12.352,5816.0,3.63,-2.06,1.45,Abohalima,NaN,NaN,6.64,NaN,NaN,NaN,NaN,5.74,3.4,NaN,NaN,NaN,NaN,4.56,1.14,3.27,NaN,3.61,2.88,5.44,2.98,4.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.89,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N